# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

### 1. Start by creating a new Notebook for this assignment.

[Link to Jupyter Notebook file.](https://github.com/jeff-mos-def/Coursera_Capstone/blob/master/Applied%20Data%20Science%20Capstone/Week%203/Peer%20graded%20Assignment%2C%20Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb)

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


### 2. Use the Notebook to build this code: 

Scrape the following Wikipedia page, [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![Canadian Neighborhoods](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1587859200000&hmac=HfAJW7cKer9DR4eIizr-tozubLdriY7H9wpJHW3oulY)

**Note**: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use *pandas* to read the table into a *pandas* dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: [http://beautiful-soup-4.readthedocs.io/en/latest/](http://beautiful-soup-4.readthedocs.io/en/latest/)

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: [https://www.youtube.com/watch?v=ng2o98k983k](https://www.youtube.com/watch?v=ng2o98k983k)

Use *pandas*, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above *pandas* dataframe.

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [8]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,", \n,"
1,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n, Malvern / ..."
2,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
3,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
4,M1G\n,Scarborough\n,"Woburn, Woburn\n, Woburn"


### 3. To create the above dataframe:

   - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
   - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
   - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
   - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
   - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
   - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### 4. Submit a link to your Notebook on your Github repository. (**10 marks**)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: [http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/](http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/), so we will use the Geocoder Python package instead: [https://geocoder.readthedocs.io/index.html](https://geocoder.readthedocs.io/index.html).

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

```
import geocoder # import geocoder
#initialize your variable to None
lat_lng_coords = None
#loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data)

Use the Geocoder package or the csv file to create the following dataframe:

![Extended Neighborhoods](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1587859200000&hmac=Bowni1LgZLBd0_UKRMoZA0TqRVq8rF3L7WhM1UbyqIs)

**Important Note**: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (**2 marks**)

### 5. Explore and cluster the neighborhoods in Toronto. 

You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (**3 marks**)